In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, AutoModelForSequenceClassification

2023-03-11 02:32:57.410197: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 02:33:00.894709: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-11 02:33:00.894746: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-11 02:33:03.877006: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

## Using a pre-trained models and pipelines

In [5]:
# https://huggingface.co/docs/transformers/quicktour

In [ ]:
sentiment_classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [4]:
sentiment_classifier("Happy happy sad sad")

[{'label': 'POSITIVE', 'score': 0.9997779726982117}]

In [17]:
sentiment_classifier("This is very easy.")

[{'label': 'POSITIVE', 'score': 0.9978420734405518}]

In [19]:
generator = pipeline(task="text-generation")

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [21]:
generator('Pluralsight')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Pluralsight, who will work with the City of Calgary to promote a "cultural tourism" model where local businesses will provide transportation, accommodation and entertainment to the community.\n\nThe Calgary Police Service has partnered with the local businesses in St.'}]

In [22]:
generator('Machine Learning will')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Machine Learning will provide you real-time, low-cost, and non-monotonic data for your Google Apps. We'll be using the same deep learning model as Google App Dev which we already have in the App Dev repository and will use"}]

In [24]:
generator('Hablas otro idioma?')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hablas otro idioma?\n\nAh! (cough, oatmeal) What's my name?\n\nHablas otro, otro? (Coughing) Oh, my! I wonder who he is..."}]

## Custom model and tokenizer Hub

In [71]:
model_path = "nlptown/bert-base-multilingual-uncased-sentiment"

In [72]:
model = TFAutoModelForSequenceClassification.from_pretrained(model_path)

Some layers from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [73]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [74]:
tokenizer.tokenize('SOMETIMES YOU’VE GOTTA RUN BEFORE YOU CAN WALK.')

['sometimes',
 'you',
 '[UNK]',
 've',
 'gotta',
 'run',
 'before',
 'you',
 'can',
 'walk',
 '.']

In [75]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [76]:
classifier("SOMETIMES YOU’VE GOTTA RUN BEFORE YOU CAN WALK.")

[{'label': '3 stars', 'score': 0.3487014174461365}]

## Zero Shot Classification

In [2]:
zero_shot_model_path = 'facebook/bart-large-mnli'

In [3]:
zero_shot_model = AutoModelForSequenceClassification.from_pretrained(zero_shot_model_path)

In [6]:
tokenizer_zero = AutoTokenizer.from_pretrained(zero_shot_model_path)

In [20]:
classifier_zero_shot("Very helpful guides and videos, easy to follow and very thorough! Keep it up!", ["positive", "negative"])

{'sequence': 'Very helpful guides and videos, easy to follow and very thorough! Keep it up!',
 'labels': ['positive', 'negative'],
 'scores': [0.9897369742393494, 0.010262986645102501]}

In [21]:
classifier_zero_shot("""
i would recommand using other learning methods not just videos because when i tried to 
implement it in real world scenarios it wasn't really helpful
""", ["positive", "negative"])

{'sequence': "\ni would recommand using other learning methods not just videos because when i tried to \nimplement it in real world scenarios it wasn't really helpful\n",
 'labels': ['negative', 'positive'],
 'scores': [0.8713861107826233, 0.1286139041185379]}

In [8]:
classifier_zero_shot = pipeline("zero-shot-classification", model=zero_shot_model, tokenizer=tokenizer_zero)

In [14]:
sequence_to_classify_1 = "Always code as if the guy who ends up maintaining your code will be a violent psychopath who knows where you live"
candidate_labels_1 = ['programming', 'cooking', 'dancing']

In [15]:
classifier_zero_shot(sequence_to_classify_1, candidate_labels_1)

{'sequence': 'Always code as if the guy who ends up maintaining your code will be a violent psychopath who knows where you live',
 'labels': ['programming', 'dancing', 'cooking'],
 'scores': [0.9860000014305115, 0.007130641955882311, 0.006869337987154722]}

In [16]:
sequence_to_classify_2 = """
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")
"""
candidate_labels_2 = ['python', 'javascript', 'machine-learning', 'dancing', 'running', 'programming']

In [17]:
classifier_zero_shot(sequence_to_classify_2, candidate_labels_2)

{'sequence': '\nfrom transformers import pipeline\nclassifier = pipeline("zero-shot-classification",\n                      model="facebook/bart-large-mnli")\n',
 'labels': ['machine-learning',
  'programming',
  'running',
  'javascript',
  'python',
  'dancing'],
 'scores': [0.39287713170051575,
  0.33438244462013245,
  0.18231745064258575,
  0.034957822412252426,
  0.028216687962412834,
  0.02724849246442318]}